# Developing Data Understanding

## In this notebook, we demonstrate preliminary analysis needed to understand the data

In [1]:
import pandas as pd
import sqlite3

In [2]:
src_path = "drive/MyDrive/dlsu_workshop"

## Setup Simulated Source Databases
We will use our source csv files to simulate the source tables that we need to access in this workshop.

In [20]:
# Create a connection to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# 'topup_transactions' table
tbl_topups = pd.read_csv(f"{src_path}/fake_data/topup_transactions.csv")
tbl_topups.to_sql('topup_transactions', conn, index=False, if_exists='replace')

# 'customer_profile' table
tbl_profiles = pd.read_csv(f"{src_path}/fake_data/customer_profile.csv")
tbl_profiles.to_sql('customer_profile', conn, index=False, if_exists='replace')

# 'usage_summary' table
tbl_usage = pd.read_csv(f"{src_path}/fake_data/usage_summary.csv")
tbl_usage.to_sql('usage_summary', conn, index=False, if_exists='replace')

# 'promo_registration' table
tbl_promos = pd.read_csv(f"{src_path}/fake_data/promo_registration.csv")
tbl_promos.to_sql('promo_registration', conn, index=False, if_exists='replace')

46764

## Test DB Access

In [4]:
# Test Connection to Topup table
q_profiles = """
SELECT *
FROM customer_profile
LIMIT 5
"""

df_profiles = pd.read_sql_query(q_profiles, conn)
df_profiles.head()

,user_id,month,segment,tenure_months,tenure_class,registration_date,device_type,gender,age
0,U0000000,2024_07,01_new_user,4,01_new,2024-03-01,feature_phone,female,38
1,U0000000,2024_08,01_new_user,5,01_new,2024-03-01,feature_phone,female,38
2,U0000000,2024_09,01_new_user,6,01_new,2024-03-01,feature_phone,female,38
3,U0000000,2024_10,01_new_user,7,02_post_onboarding,2024-03-01,feature_phone,female,38
4,U0000000,2024_11,01_new_user,8,02_post_onboarding,2024-03-01,feature_phone,female,38


In [5]:
# Test Connection to Topup table
q_topups = """
SELECT *
FROM topup_transactions
LIMIT 5
"""

df_topups = pd.read_sql_query(q_topups, conn)
df_topups.head()

,user_id,month,topup_date,topup_amount,channel
0,U0000000,2024_07,2024-07-08,100,app
1,U0000001,2024_07,2024-07-23,20,app
2,U0000003,2024_07,2024-07-03,50,retailer
3,U0000003,2024_07,2024-07-24,20,retailer
4,U0000004,2024_07,2024-07-28,20,retailer


In [6]:
# Test Connection to Usage table
q_usage = """
SELECT *
FROM usage_summary
LIMIT 5
"""

df_usage = pd.read_sql_query(q_usage, conn)
df_usage.head()

,user_id,month,data_mb,voice_minutes,sms_count,p_churn
0,U0000000,2024_07,174.84,0,0,0.220122
1,U0000000,2024_08,69.94,0,4,0.293753
2,U0000000,2024_09,69.94,0,0,0.266630
3,U0000000,2024_10,48.96,0,0,0.268568
4,U0000000,2024_11,34.27,0,0,0.277022


In [7]:
# Test Connection to Promo Registration table
q_promos = """
SELECT *
FROM promo_registration
LIMIT 5
"""

df_promos = pd.read_sql_query(q_promos, conn)
df_promos.head()

,user_id,month,registration_date,promo_type
0,U0000006,2024_07,2024-07-04,social_bundle
1,U0000006,2024_07,2024-07-24,weekend_special
2,U0000007,2024_07,2024-07-12,combo_pack
3,U0000007,2024_07,2024-07-06,combo_pack
4,U0000023,2024_07,2024-07-21,data_booster


## Data Quality Checks
For this segment, we will use SQL to perform the checks rather than pandas. This is because in reality, we might have very large databases that might be too expensive and/or time-consuming to process outside of the SQL databases.

### Row, Null Counts, DISTINCT subcriber counts, Min and Max Dates

We will begin with the customer profile table. For this demo, this table will also serve as our reference for the actual number of active subscribers

In [8]:
q_profile_counts = """
SELECT
    COUNT(*) AS row_count,
    COUNT(DISTINCT user_id) AS distinct_users,
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS null_id,
    SUM(CASE WHEN segment IS NULL THEN 1 ELSE 0 END) AS null_segment,
    SUM(CASE WHEN registration_date IS NULL THEN 1 ELSE 0 END) AS null_date,
    SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS null_age,
    SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS null_gender,
    SUM(CASE WHEN device_type IS NULL THEN 1 ELSE 0 END) AS null_device
FROM customer_profile;
"""

pd.read_sql_query(q_profile_counts, conn)

,row_count,distinct_users,null_id,null_segment,null_date,null_age,null_gender,null_device
0,115950,21930,0,0,0,0,0,0


From above, we see that we have 20k+ unique customers with 0 nulls. Notice that each row corresponds to a unique subscriber/user which is the expected behavior for a profile table.

Note:
1. In real life, we might see nulls more often and that necessitates a discussion with data owners or analysts to see if it is expected.
2. Ideally though, the customer_profile should be clean as it has undergone preliminary data quality checks from data engineers and data quality teams prior to serving it to analytics teams.
3. However, it is still a good practice to perform an independent check prior to building models on top of data.

In [21]:
# topups
q_topup_counts = """
SELECT
    COUNT(*) AS row_count,
    COUNT(DISTINCT user_id) AS distinct_users,
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS null_user_id,
    SUM(CASE WHEN topup_date IS NULL THEN 1 ELSE 0 END) AS null_topup_date,
    SUM(CASE WHEN topup_amount IS NULL THEN 1 ELSE 0 END) AS null_topup_amount,
    SUM(CASE WHEN channel IS NULL THEN 1 ELSE 0 END) AS null_channel,
    MIN(topup_date) AS min_topup_date,
    MAX(topup_date) AS max_topup_date
FROM topup_transactions;
"""

pd.read_sql_query(q_topup_counts, conn)

,row_count,distinct_users,null_user_id,null_topup_date,null_topup_amount,null_channel,min_topup_date,max_topup_date
0,146799,21179,0,0,0,0,2024-07-01,2025-06-28


We see that the topup transactions table is also clean in terms of null counts. Note however that the distinct users are less than the total active customers. This means that some users did not have any usage during this period

In [22]:
# usage
q_usage_counts = """
SELECT
    COUNT(*) AS row_count,
    COUNT(DISTINCT user_id) AS distinct_users,
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS null_user_id,
    SUM(CASE WHEN month IS NULL THEN 1 ELSE 0 END) AS null_usage_month,
    SUM(CASE WHEN data_mb IS NULL THEN 1 ELSE 0 END) AS null_data_mb,
    SUM(CASE WHEN voice_minutes IS NULL THEN 1 ELSE 0 END) AS null_voice_minutes,
    SUM(CASE WHEN sms_count IS NULL THEN 1 ELSE 0 END) AS null_sms_count,
    MIN(month) AS min_usage_date,
    MAX(month) AS max_usage_date
FROM usage_summary;
"""

pd.read_sql_query(q_usage_counts, conn)

,row_count,distinct_users,null_user_id,null_usage_month,null_data_mb,null_voice_minutes,null_sms_count,min_usage_date,max_usage_date
0,115950,21930,0,0,0,0,0,2024_07,2025_06


Again, we note that there are no null values in the usage table. We also note that all users have records in the usage table

In [23]:
# promo registration
q_promo_counts = """
SELECT
    COUNT(*) AS row_count,
    COUNT(DISTINCT user_id) AS distinct_users,
    COUNT(DISTINCT promo_type) AS distinct_promos,
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS null_user_id,
    SUM(CASE WHEN registration_date IS NULL THEN 1 ELSE 0 END) AS null_registration_date,
    SUM(CASE WHEN promo_type IS NULL THEN 1 ELSE 0 END) AS null_promo_type,
    MIN(registration_date) AS min_registration_date,
    MAX(registration_date) AS max_registration_date
FROM promo_registration;
"""

pd.read_sql_query(q_promo_counts, conn)

,row_count,distinct_users,distinct_promos,null_user_id,null_registration_date,null_promo_type,min_registration_date,max_registration_date
0,46764,15517,4,0,0,0,2024-07-01,2025-06-28


Note that we have uses who did not avail of any promo. Also note that there are 5 unique promos recorded within the period.

These needs to be validated with the client and their analysis for alignment before starting any further analysis.

### Defining and Tracking Churn

Step 1: Track monthly activities across topup, usage, and promo. Create a table for this.

In [44]:
q_monthly = """
CREATE TABLE user_activity_monthly AS
WITH months AS (
  SELECT DISTINCT month FROM (
    SELECT month FROM usage_summary
    UNION
    SELECT month FROM topup_transactions
    UNION
    SELECT month FROM promo_registration
  )
),

users AS (
  SELECT DISTINCT user_id, registration_date FROM customer_profile
),

active_months AS (
  SELECT u.user_id, m.month
  FROM users u
  CROSS JOIN months m
  WHERE m.month >= strftime('%Y_%m', u.registration_date)
),

usage_agg AS (
  SELECT
    user_id,
    month,
    SUM(data_mb) AS total_data_mb,
    SUM(voice_minutes) AS total_voice_minutes,
    SUM(sms_count) AS total_sms_count,
    CASE WHEN SUM(data_mb) > 0 OR SUM(voice_minutes) > 0 OR SUM(sms_count) > 0 THEN 1 ELSE 0 END AS has_usage
  FROM usage_summary
  GROUP BY user_id, month
),

topup_agg AS (
  SELECT
    user_id,
    month,
    COUNT(*) AS topup_count,
    SUM(topup_amount) AS total_topup_amount,
    1 AS has_topup
  FROM topup_transactions
  GROUP BY user_id, month
),

promo_agg AS (
  SELECT
    user_id,
    month,
    COUNT(*) AS promo_count,
    1 AS has_promo
  FROM promo_registration
  GROUP BY user_id, month
),

joined AS (
  SELECT
    a.user_id,
    a.month,
    COALESCE(u.has_usage, 0) AS has_usage,
    COALESCE(u.total_data_mb, 0) AS total_data_mb,
    COALESCE(u.total_voice_minutes, 0) AS total_voice_minutes,
    COALESCE(u.total_sms_count, 0) AS total_sms_count,

    COALESCE(t.has_topup, 0) AS has_topup,
    COALESCE(t.topup_count, 0) AS topup_count,
    COALESCE(t.total_topup_amount, 0) AS total_topup_amount,

    COALESCE(p.has_promo, 0) AS has_promo,
    COALESCE(p.promo_count, 0) AS promo_count
  FROM active_months a
  LEFT JOIN usage_agg u ON a.user_id = u.user_id AND a.month = u.month
  LEFT JOIN topup_agg t ON a.user_id = t.user_id AND a.month = t.month
  LEFT JOIN promo_agg p ON a.user_id = p.user_id AND a.month = p.month
),

activity_flagged AS (
  SELECT *,
    CASE
      WHEN has_usage = 0 AND has_topup = 0 AND has_promo = 0 THEN 1
      ELSE 0
    END AS churned
  FROM joined
),

first_churn AS (
  SELECT user_id, MIN(month) AS first_churn_month
  FROM activity_flagged
  WHERE churned = 1
  GROUP BY user_id
)

SELECT
  a.user_id,
  a.month,
  a.has_usage,
  a.total_data_mb,
  a.total_voice_minutes,
  a.total_sms_count,
  a.has_topup,
  a.topup_count,
  a.total_topup_amount,
  a.has_promo,
  a.promo_count,
  a.churned
FROM activity_flagged a
LEFT JOIN first_churn f
  ON a.user_id = f.user_id AND a.month = f.first_churn_month
WHERE a.churned = 0 OR f.first_churn_month IS NOT NULL
ORDER BY a.user_id, a.month;
"""

# drop table if it already exists
q_drop_table = """
DROP TABLE IF EXISTS user_activity_monthly;
"""

conn.execute(q_drop_table)
conn.execute(q_monthly)

In [45]:
# test connection to user_activity_monthly
q_monthly_test = """
SELECT *
FROM user_activity_monthly
WHERE churned = 1
"""

pd.read_sql(q_monthly_test, conn)

,user_id,month,has_usage,total_data_mb,total_voice_minutes,total_sms_count,has_topup,topup_count,total_topup_amount,has_promo,promo_count,churned
0,U0000000,2025_02,0,0.0,0,0,0,0,0,0,0,1
1,U0000001,2024_10,0,0.0,0,0,0,0,0,0,0,1
2,U0000002,2024_09,0,0.0,0,0,0,0,0,0,0,1
3,U0000003,2024_10,0,0.0,0,0,0,0,0,0,0,1
4,U0000004,2024_09,0,0.0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12885,U0020883,2025_06,0,0.0,0,0,0,0,0,0,0,1
12886,U0020885,2025_06,0,0.0,0,0,0,0,0,0,0,1
12887,U0020903,2025_06,0,0.0,0,0,0,0,0,0,0,1
12888,U0020910,2025_06,0,0.0,0,0,0,0,0,0,0,1


In [46]:
# how many churned monthly
q_monthly_churn_counts = """
SELECT month, COUNT(*) AS count
FROM user_activity_monthly
WHERE churned = 1
GROUP BY month
"""

pd.read_sql(q_monthly_churn_counts, conn)

,month,count
0,2024_07,3
1,2024_08,1417
2,2024_09,1302
3,2024_10,1346
4,2024_11,1227
5,2024_12,1186
6,2025_01,1173
7,2025_02,1138
8,2025_03,1038
9,2025_04,1068


We notice that the count for the first month is different. This could be due to lack of information of which users were already there in the prior months.

This means that we can skip the first month.

We can then relay this information to our stakeholders before we proceed with the next round of EDA.